In [3]:
from typing import Annotated  # for type hints bhai kaisa hona chahiye var
from typing_extensions import TypedDict #dictionary values data types



##**Using GRAPH API**


**State**: A shared data structure that represents the current snapshot of your application. It can be any data type, but is typically defined using a shared state schema.

**Nodes**: Functions that encode the logic of your agents. They receive the current state as input, perform some computation or side-effect, and return an updated state.

**Edges**: Functions that determine which Node to execute next based on the current state. They can be conditional branches or fixed transitions.

In [4]:
from langgraph.graph import StateGraph , START , END #stategraph is the major class
from langgraph.graph.message import add_messages #reducers ---> APPEND MASSEGES INSTEAD OF REPLACING

In [5]:
#Our State is a dictionary that must have one key: messages.”
class State(TypedDict):
   #Messages have the type "list". The 'add_messages' function in the
   # annotation defines how this state key should be updated
   #in this case it appends messages to list , rather than overwritting them
   messages:Annotated[list , add_messages]
   #That add_messages function tells LangGraph how to update this key
   #whenever new info comes in. Without it, LangGraph would just overwrite the list
   #every time something changes.With it, LangGraph knows to append new messages instead.


In [6]:
!pip install langchain langgraph langsmith langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.1 MB/s eta 0:00:00


In [9]:
from google.colab import userdata
import os

api_key = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = api_key


In [19]:
from langchain.chat_models import init_chat_model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [13]:
from langchain.chat_models import init_chat_model
gemini_15 = init_chat_model("gemini-2.5-flash", model_provider="google_genai", temperature=1)
gemini_15.invoke("what's your name")

AIMessage(content='I do not have a name. I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c1f56-5f8c-7b23-b93b-1e006c59b101-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 6, 'output_tokens': 275, 'total_tokens': 281, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 257}})

In [14]:
!pip install -U langgraph

## Node Function

In [15]:
def chatbot(state:State):
  return { "messages" : [llm.invoke(state["messages"])]}



## Creating Graph

In [16]:
#initialize graph
graph_builder=StateGraph(State)

#create graph
graph_builder.add_node("LLM", chatbot)  #chatbot node
graph_builder.add_edge(START , "LLM")   #start--->LLM
graph_builder.add_edge("LLM" , END)     #llm ---> END

#complie graph
graph = graph_builder.compile()

## Invoke graph

In [20]:
response = graph.invoke({"messages": "hi"})

In [21]:
response

{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='2feb9b90-2113-4a78-9427-f2736bb2b5b7'),
  AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c1f58-af5f-74b0-b616-f33450146844-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2, 'output_tokens': 368, 'total_tokens': 370, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 358}})]}

In [22]:
response['messages'][-1].content

'Hi there! How can I help you today?'

In [23]:
for x in graph.stream({'messages' : "Why claude is better than gpt at coding tasks . Limit your answers to 40 words. "} , stream_mode = ['updates' , 'values']) :
  print("-----------------")
  print(x)
  # print("---- VALUES ----")
  # print(x.get("values"))

  # print("---- UPDATES ----")
  # print(x.get("updates"))
  # for v in x.values() :
  #   print("********************")
  #   print(v)

-----------------
('values', {'messages': [HumanMessage(content='Why claude is better than gpt at coding tasks . Limit your answers to 40 words. ', additional_kwargs={}, response_metadata={}, id='551fe50f-dc8a-4e2c-8a5d-9899df9cf48f')]})
-----------------
('updates', {'LLM': {'messages': [AIMessage(content='Claude often offers a larger context window for complex projects, enabling better logical reasoning and superior instruction following. This leads to more accurate, robust, and contextually aware code solutions, often outperforming GPT in specific coding benchmarks.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c1f58-f610-7032-a6c5-ffbd9c7c080b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 23, 'output_tokens': 702, 'total_tokens': 725, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 658}})]}})
----